<a href="https://colab.research.google.com/github/fabian692/DNN-and-preprocesin-cicids-2017/blob/master/DNN%20and%20preprocessing%20cicids%202017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install pyspark


In [ ]:
pip install findspark

In [ ]:
pip install sklearn

In [ ]:
import numpy as np
import requests
import time
import tensorflow as tf
import keras
import random
import numpy as np
import sklearn
import threading
import requests
import pyspark
import findspark

from keras.optimizers import *
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors



from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

In [ ]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [ ]:
# preprocesamiento de datos
InD = np.zeros((0,79),dtype=object)
for x in glob("/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/*csv"):
    print(x)
    InD=np.vstack((InD,pd.read_csv(x)))

/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
/content/drive/My Drive/Colab Notebooks/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv


In [ ]:
Dt=InD[:,:-1].astype(float)

In [ ]:
#Remove nan values
LNMV=InD[~np.isnan(Dt).any(axis=1),-1]
DtNMV=Dt[~np.isnan(Dt).any(axis=1)]
#Remove Inf values
LNMIV=LNMV[~np.isinf(DtNMV).any(axis=1)]
DtNMIV=DtNMV[~np.isinf(DtNMV).any(axis=1)]

In [ ]:
del(DtNMV)

In [ ]:
np.save('NBx', MinMaxScaler().fit_transform(DtNMIV))
np.save('NBy', (LNMIV=='BENIGN').astype(int))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",(LNMIV=='BENIGN').astype(int).reshape(-1,1))

In [ ]:
DtNMIV.shape

(2827876, 78)

In [ ]:
MCDt=DtNMIV[LNMIV!='BENIGN',:]

In [ ]:
MCDt.shape

(556556, 78)

In [ ]:
MCL=LNMIV[LNMIV!='BENIGN']

In [ ]:
MCL.shape

(556556,)

In [ ]:
LNMIV.shape,MCL.shape

((2827876,), (556556,))

In [ ]:
np.save('NMCx', MinMaxScaler().fit_transform(MCDt))
np.save('NMCy', LabelEncoder().fit_transform(MCL))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",OneHotEncoder(sparse=False).fit_transform(MCL.reshape(-1,1)))

In [ ]:
#Replace missing values with average and inf values with max
A14=np.average(DtNMIV[:,14])
A15=np.average(DtNMIV[:,15])
M14=np.max(DtNMIV[:,14])
M15=np.max(DtNMIV[:,15])
for c in range(Dt.shape[0]):
    if np.isnan(Dt[c,14]):
        Dt[c,14]=A14
    if np.isnan(Dt[c,15]):
        Dt[c,15]=A15
    if np.isinf(Dt[c,14]):
        Dt[c,14]=M14
    if np.isinf(Dt[c,15]):
        Dt[c,15]=M15

In [ ]:
np.save('RBx', MinMaxScaler().fit_transform(Dt))
np.save('RBy', (InD[:,-1]=='BENIGN').astype(int))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",(InD[:,-1]=='BENIGN').astype(int).reshape(-1,1))

In [ ]:
MCDt=Dt[InD[:,-1]!='BENIGN',:]

In [ ]:
MCDt.shape

(557646, 78)

In [ ]:
MCL=InD[InD[:,-1]!='BENIGN',-1]

In [ ]:
MCL.shape

(557646,)

In [ ]:
np.save('RMCx', MinMaxScaler().fit_transform(MCDt))
np.save('RMCy', LabelEncoder().fit_transform(MCL))
np.save("/content/drive/My Drive/Colab Notebooks/preprocesamiento/",OneHotEncoder(sparse=False).fit_transform(MCL.reshape(-1,1)))

In [ ]:
# Load the dataset and labels new--
x=np.load('/content/NBx.npy')
y=np.load('/content/NBy.npy')
x.shape,y.shape

((2827876, 78), (2827876,))

In [ ]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [ ]:
# Calculate the rank of each feature
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics.cluster import homogeneity_score
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0])
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
   # r=homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    #R.append(r)




KeyboardInterrupt: ignored

In [ ]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [ ]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [ ]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"])
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"])
        model = Sequential()
        model.add(Dense(128,input_dim=fd.shape[1],activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(64,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(32,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(1,activation='sigmoid',use_bias=True)) #The number of neurons is equal to the number of classes
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
        model.summary()
        model.fit(fd[train],y[train] , epochs=1000)
 
# evaluamos el modelo
scores = model.evaluate(fd[train],y[train])
 
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print (model.predict(fd[train]).round())
        #trainer = SingleTrainer(keras_model=model, worker_optimizer='adam', loss='binary_crossentropy', num_epoch=1000)
        #trained_model = trainer.train(TrD)
        #trainer = AEASGD(keras_model=model, worker_optimizer='adam,', loss='binary_crossentropy', num_workers=num_workers, 
                 #batch_size=32, features_col="features_normalized", label_col="label", num_epoch=1000,
                 #communication_window=32, rho=5.0, learning_rate=0.1)
       # trainer.set_parallelism_factor(1)
       # trained_model = trainer.train(TrD)
        #st = time.time()
        #predictor = ModelPredictor(keras_model=trained_model)
        #et=time.time()-st
        #ff=predictor.predict(TsD)
        #ts=np.array(map(lambda x: x[0],ff.select('prediction').collect())).reshape(-1,1)
        #pp=pp+metrics.accuracy_score(y[test].reshape(-1,1),(ts>=0.5).astype(int))
        #lpa=np.vstack((lpa,np.hstack((y[test].reshape(-1,1),ts))))
    #pp=pp/kfolds.n_splits
    #np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    #smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time required to predict a label per each object in uS.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3

KeyboardInterrupt: ignored

In [ ]:
smr